### Imports

In [67]:
from random import random
from random import seed
from random import randint
import os
import numpy as np

### Score Metrics: TOS and ADS

In [23]:
def score_metrics(scores, nbooks, signup, dailyLimit, daysLeft): 
    # total offered score (TOS): the max score that the library can provide in the remaining days including signup
    # average daily score (ADS): approx daily offering based on TOS
    # scores: sorted list of remaining books
    # nbooks: number of books
    # signup: signup time in days
    # dailylLimit: number of books scanned daily
    # daysLeft: number of remaining days
    
    # reqDays: days required to scan all books
    # nleftover: extra books that cannot be scanned
    
    if signup >= daysLeft:
        return 0,0 
    
    totalScore = sum(scores)
    reqDays = nbooks/dailyLimit
    scanDays = daysLeft - signup
    
    nleftover = nbooks - (dailyLimit * scanDays)
    
    if nleftover < 0:
        nleftover = 0
    
    tos = sum(scores[:len(scores)-nleftover])
    
    if scanDays >= reqDays:
        ads = tos/reqDays
    else:
        ads = tos/scanDays
        
    
    return tos,ads    


### Class Definitions

In [268]:
class Library():
    
    def __init__(self,**kwargs):
        self.name = kwargs.pop('name')
        self.signup = kwargs.pop('signup')
        self.dailyLimit = kwargs.pop('dailyLimit')
        self.nbooks = len(scores)
        self.scores = kwargs.pop('scores')
        self.totalScore = sum(scores)
        self.tos = 0
        self.ads = 0        
        self.credit = 0
        return
    
    def getScoreMetrics(self,**kwargs):
        self.tos,self.ads = score_metrics(self.scores, self.nbooks, self.signup, self.dailyLimit, daysLeft)
        return
    
    def showDetails(self, name=''):
        print('{}:\n nbooks: {}\n Sum: {}\n Daily Limit: {}\n Signup: {}\n TOS: {}\n ADS: {}\n Credit: {}\n Scores: {}\n\n'.format(self.name,self.nbooks,self.totalScore,self.dailyLimit,self.signup,self.tos,self.ads,self.credit,self.scores))

### Testing Functions

In [184]:
def generateBookScores(totalScore, nbooks):
    # totalScore: the max score of book list
    # nbooks: the number of books in the list
    
    # scoreList: final scores of argument specifications
    
    scoreList = []
    runningTotal = totalScore
    runningIndex = nbooks
    
    for i in range(nbooks):
        initialLimit = runningTotal - runningIndex
        score = randint(1,initialLimit)
        scoreList.append(score)
        runningTotal -= score
        runningIndex -= 1
    
    extra = totalScore - sum(scoreList)
    scoreList[randint(0,nbooks-1)] += extra
    
    scoreList.sort(reverse=True)
    return scoreList

scores = generateBookScores(300,12)
print(' Score List: {}\n Sum: {}\n'.format(scores,sum(scores)))

 Score List: [238, 31, 10, 4, 4, 4, 3, 2, 1, 1, 1, 1]
 Sum: 300



### Driver

In [269]:
nbooks = 12
scores = generateBookScores(30,nbooks)
signup = 6
dailyLimit = 2
daysLeft = 6
name = 'L1'

L1 = Library(nbooks=nbooks, scores=scores, signup=signup, dailyLimit=dailyLimit, name=name)
L1.getScoreMetrics(daysLeft=daysLeft)
L1.showDetails()

L1:
 nbooks: 12
 Sum: 30
 Daily Limit: 2
 Signup: 6
 TOS: 0
 ADS: 0
 Credit: 0
 Scores: [8, 7, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1]




### Input (original)

In [52]:
# f = open('a_example.txt','r')
# f = f.read().split('\n')

# for i in range(len(f)):
#     f[i] = list(map(int,f[i].split()))
# print(f)
# no_books = f[0][0]
# no_libraries = f[0][1]
# no_days = f[0][2]
# no_days_left = f[0][2]
# scores = f[1]

# libraries = []
# for i in range(0,no_libraries*2,2):
#     libraries.append(library(f[2+i][0],f[2+i][1],f[2+i][2],f[3+i]))

# len(libraries)

[[6, 2, 7], [1, 2, 3, 6, 5, 4], [5, 2, 2], [0, 1, 2, 3, 4], [4, 3, 1], [0, 2, 3, 5], []]


2